# 目錄
- [關於 Guidance](../../../../code/01.Introduce)
- [設定](../../../../code/01.Introduce)
- [無約束生成](../../../../code/01.Introduce)
- [Phi 3 的表達](../../../../code/01.Introduce)
- [正則表達式](../../../../code/01.Introduce)
- [選擇](../../../../code/01.Introduce)
- [思維鏈](../../../../code/01.Introduce)
- [JSON 生成](../../../../code/01.Introduce)
- [HTML 生成](../../../../code/01.Introduce)


# 關於 Guidance
Guidance 是一個經過驗證的開源 Python 庫，用於控制任何語言模型 (LM) 的輸出。只需一次 API 調用，您就可以在 Python 中表達模型必須遵循的精確程式化約束，並生成 JSON、Python、HTML、SQL 或任何使用案例所需的結構化輸出。

Guidance 與傳統的提示技術不同。它通過在推理層逐字元引導模型來強制執行約束，從而生成更高質量的輸出，並在高度結構化場景中使用時降低成本和延遲高達 30–50%。

想了解更多有關 Guidance 的資訊，請訪問 [GitHub 上的公共倉庫](https://github.com/guidance-ai/guidance) 或觀看 Microsoft Build 的 [Guidance 分組會議](https://www.youtube.com/watch?v=qXMNPVVlCMs)。


# 設置
1. 使用 `pip install guidance --pre` 安裝 Guidance
2. 在 Azure 部署 Phi 3.5 mini 端點，前往 https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml 並點擊「部署」按鈕
3. 將端點的 API 金鑰存儲在名為 `AZURE_PHI3_KEY` 的環境變數中，並將 URL 存儲在名為 `AZURE_PHI3_URL` 的環境變數中


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# 無限制生成
可以使用 `gen()` 函數生成不受限制的文本。這與不使用 Guidance 的模型效果相同。

## 聊天格式
與許多聊天模型類似，Phi-3 期望用戶與助手之間的消息以特定格式呈現。Guidance 支援 Phi-3 的聊天模板，並會為您管理聊天格式。要創建聊天回合，請將對話的每一部分放在 `with user()` 或 `with assistant()` 區塊中。可以使用 `with system()` 區塊來設置系統消息。


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## 節省 Token
在高度結構化的情境中，Guidance 可以跳過不必要的 token，只生成必要的 token，從而提升性能、提高效率並節省 API 成本。生成的 token 在此筆記本中會以高亮背景顯示。強制生成的 token 則不會高亮顯示，其成本與輸入 token 相同，估計約為輸出 token 成本的三分之一。

*注意：* 第一個無約束生成的範例無法強制生成任何 token，因為我們未提供任何限制。


# 為 Phi 3 發聲  
透過 Guidance，您可以輕鬆地將文字注入模型的回應中。如果您希望引導模型的輸出朝特定方向發展，這將非常有幫助。


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# 使用正則表達式進行限制
在前面的例子中，Phi 3 在回答問題「Canberra」後，提供了後續的解釋。為了將模型的輸出限制為精確的一個字，可以使用正則表達式。


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# 從多個選項中選擇
當已知一些可能的選項時，可以使用 `select()` 函數讓模型從選項列表中進行選擇。


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

使用 `select()`，僅生成了標記 `Can`。由於 `Canberra` 是唯一可能完成回應的選項，其餘的標記被強制生成。


# 思維鏈
思維鏈是一種技術，可以幫助提升模型輸出的品質，透過鼓勵模型逐步處理問題來達成。通常，要得到最終答案需要多次提示。首先，指示模型逐步思考。接著，再次提示模型提供最終答案。使用標準的聊天推理 API 時，這需要進行兩次 API 呼叫，而模型生成的「思維鏈」會被收費兩次——一次是模型生成時作為輸出字元，另一次是第二次呼叫時作為輸入字元。然而，使用 Guidance，整個多步驟的過程會在單次 API 呼叫中完成並計費，從而降低成本和延遲。


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON 生成
Guidance 可用於確保生成符合 JSON 架構或 pydantic 模型的 JSON，例如這裡顯示的使用者檔案架構。


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML 生成

Guidance 也可以用來生成程式碼，並遵循程式語言的語法要求。在本節中，我們將建立一個簡單的 Guidance 程式，用於撰寫非常簡單的 HTML 網頁。

我們會將網頁拆分成較小的部分，每個部分都有自己的 Guidance 函數。然後將這些部分結合在最終的函數中，以生成一個 HTML 網頁。
接著，我們會在 Azure AI 中執行這個函數，並使用支援 Guidance 的模型。

*注意：* 這不會是一個功能完整的 HTML 生成器；目的是展示如何根據個人需求創建結構化的輸出。

首先，我們從 Guidance 中匯入所需的內容：


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML 網頁具有高度結構化，我們將使用 Guidance 來「強制」頁面中的這些部分。  
當我們明確要求模型提供文字時，我們需要確保它不包含任何可能是標籤的內容——也就是說，我們必須排除「<」和「>」字符：


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

然後我們可以使用此函數在任意的 HTML 標籤內生成文本：


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

現在，讓我們建立頁面標題。  
作為其中的一部分，我們需要生成一個頁面標題：


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML 頁面的主體將會包含標題和段落。  
我們可以定義一個函數來完成每個部分：


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

現在，定義 HTML 主體的函數。  
這裡使用 `select()` 並設置 `recurse=True` 來生成多個標題和段落：


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

接下來，我們來到生成完整 HTML 頁面的函數。  
我們先加入 HTML 開始標籤，接著生成標頭，再生成主體，最後添加結束的 HTML 標籤：


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

我們提供了一個使用者友善的封裝，這將允許我們：
- 設定生成的溫度
- 從 Model 物件中捕捉生成的頁面


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

然後我們可以將輸出寫入檔案：


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

並[查看結果](../../../../code/01.Introduce/sample_page.html)。



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
